In [47]:
import kagglehub
import os
import shutil

dataset_ref = "ckay16/accident-detection-from-cctv-footage"

datasets_path = '../../datasets/'

path = kagglehub.dataset_download(handle = dataset_ref)

os.makedirs(datasets_path, exist_ok=True)

dataset_name = os.path.basename(path)
shutil.move(path, os.path.join(datasets_path, dataset_name))

print(path)

100%|██████████| 249M/249M [00:08<00:00, 29.7MB/s] 

Extracting files...


/Users/ddsharafiev/.cache/kagglehub/datasets/ckay16/accident-detection-from-cctv-footage/versions/1


In [48]:
import gdown
import zipfile

file_id = '1AcGl5nyFoTmqNklxk-xpy2rb7tZT9EwL'
output_path = '../../datasets/CADP/'
zip_path = os.path.join(output_path, 'cadp.zip')

os.makedirs(output_path, exist_ok=True)

gdown.download(id=file_id, output=zip_path, quiet=False)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_path)

print("CADP dataset downloaded to:", zip_path)


Downloading...
From (original): https://drive.google.com/uc?id=1AcGl5nyFoTmqNklxk-xpy2rb7tZT9EwL
From (redirected): https://drive.google.com/uc?id=1AcGl5nyFoTmqNklxk-xpy2rb7tZT9EwL&confirm=t&uuid=3bc5dd16-a5a3-419e-b63f-605aa9173c8f
To: /Users/ddsharafiev/innopolis_projects/road-accident-detection/datasets/CADP/cadp.zip
100%|██████████| 46.6M/46.6M [00:02<00:00, 21.4MB/s]


CADP dataset downloaded to: ../../datasets/CADP/cadp.zip


In [49]:
import re

mask = r'.*\.jpg'
pattern = re.compile(mask)

In [50]:
filenames = os.listdir('../../datasets/CADP/')

for filename in filenames:
    if not pattern.match(filename):
        full_path = os.path.join('../../datasets/CADP/', filename)
        if os.path.isdir(full_path):
            shutil.rmtree(full_path)
        elif not pattern.match(filename):
            os.remove(full_path)

In [51]:
import kagglehub, os, shutil

In [52]:
dataset_ref = "ashfakyeafi/road-vehicle-images-dataset"

datasets_path = '../../datasets/road-vehicle-images/'

path = kagglehub.dataset_download(handle = dataset_ref)

os.makedirs(datasets_path, exist_ok=True)

dataset_name = os.path.basename(path)
shutil.move(path, os.path.join(datasets_path, dataset_name))

print(path)

100%|██████████| 115M/115M [00:04<00:00, 26.1MB/s] 

Extracting files...


/Users/ddsharafiev/.cache/kagglehub/datasets/ashfakyeafi/road-vehicle-images-dataset/versions/2


In [53]:
dataset_ckay16_path = '../../datasets/1/data/'

non_accident_images_path = '../../datasets/road-vehicle-images/2/trafic_data/train/images/'
accident_images_path = '../../datasets/CADP/'

dataset_train_path = '../../datasets/train/'
dataset_val_path = '../../datasets/val/'
dataset_test_path = '../../datasets/test/'

non_accident_mask = r'Pias.*\.jpg'
accident_mask = r'.*\.jpg'

# ckay16

In [54]:
# move all from dataset_ckay16_path/test to dataset_test_path
os.makedirs(dataset_test_path, exist_ok=True)
for case in ['train', 'test', 'val']:
    for filename in os.listdir(os.path.join(dataset_ckay16_path, case)):
        full_file_path = os.path.join(dataset_ckay16_path, case, filename)
        shutil.move(full_file_path, os.path.join(eval(f'dataset_{case}_path'), filename))

# CADP

In [55]:
filegroup = {} # filename : indexes

filenames = os.listdir(accident_images_path)

for name in filenames:
    if pattern.match(name):
        filetemplate = re.sub(r'_(\d+)s\.jpg$', '', name)
        if filetemplate in filegroup:
            filegroup[filetemplate].append(name)
        else:
            filegroup[filetemplate] = [name]

for key in filegroup:
    filegroup[key].sort()

filetemplates = list(filegroup.keys())

filetemplates.sort(key=lambda x: len(filegroup[x]), reverse=True)

print(len(filegroup[filetemplates[0]]), len(filegroup[filetemplates[2]]))

33 29


In [56]:
def split_dataset_with_group(filetemplate: list[str], 
                  filegroup : dict[str, list[str]], 
                  train_ratio : float,
                  test_ratio : float) -> tuple:
    """gives filetemplates, not filenames"""
    train_set = []
    test_set = []
    val_set = []
    current_train_count = 0
    current_test_count = 0
    current_val_count = 0
    total_count = sum(len(filegroup[template]) for template in filetemplates)
    # We assume that photos are distributed randomly 
    # And they are independed based on number of photos in one group
    for template in filetemplates:
        if current_train_count / total_count < train_ratio:
            train_set.append(template)
            current_train_count += len(filegroup[template])
        elif current_test_count / total_count < test_ratio:
            test_set.append(template)
            current_test_count += len(filegroup[template])
        else:
            val_set.append(template)
            current_val_count += len(filegroup[template])
            
    return train_set, test_set, val_set, current_train_count, current_test_count, current_val_count

In [57]:
train_ratio = 0.6
test_ratio = 0.2

accident_train_set, accident_test_set, accident_val_set, train_count, test_count, val_count = split_dataset_with_group(filetemplates, 
                                                                                  filegroup, 
                                                                                  train_ratio,
                                                                                  test_ratio)

print("Train set size (in templates):", len(accident_train_set))
print("Test set size (in templates):", len(accident_test_set))
print("Val set size (in templates):", len(accident_val_set))
print("Train set size (in samples):", train_count)
print("Test set size (in samples):", test_count)
print("Val set size (in samples):", val_count)

Train set size (in templates): 31
Test set size (in templates): 67
Val set size (in templates): 84
Train set size (in samples): 262
Test set size (in samples): 87
Val set size (in samples): 84


In [58]:
# move files to train, test, val directories
for filetemplate in accident_train_set:
    for filename in filegroup[filetemplate]:
        full_file_path = os.path.join(accident_images_path, filename)
        # if there is no such file, skip it
        if not os.path.exists(full_file_path):
            continue
        shutil.move(full_file_path, os.path.join(dataset_train_path, "Accident", filename))

for filetemplate in accident_test_set:
    for filename in filegroup[filetemplate]:
        full_file_path = os.path.join(accident_images_path, filename)
        if not os.path.exists(full_file_path):
            continue
        shutil.move(full_file_path, os.path.join(dataset_test_path, "Accident", filename))

for filetemplate in accident_val_set:
    for filename in filegroup[filetemplate]:
        full_file_path = os.path.join(accident_images_path, filename)
        if not os.path.exists(full_file_path):
            continue
        shutil.move(full_file_path, os.path.join(dataset_val_path, "Accident", filename))

# road-vehicle-images

In [59]:
def split_dataset(filenames: list[str], 
                  train_ratio : float,
                  test_ratio : float) -> tuple:
    """gives filenames"""
    train_set = []
    test_set = []
    val_set = []
    current_train_count = 0
    current_test_count = 0
    current_val_count = 0

    for filename in filenames:
        if current_train_count / len(filenames) < train_ratio:
            train_set.append(filename)
            current_train_count += 1
        elif current_test_count / len(filenames) < test_ratio:
            test_set.append(filename)
            current_test_count += 1
        else:
            val_set.append(filename)
            current_val_count += 1

    return train_set, test_set, val_set, current_train_count, current_test_count, current_val_count

In [60]:
num_non_accident_images = 500

filenames = os.listdir(non_accident_images_path)[:num_non_accident_images]

print(filenames)

train_ratio = 0.6
test_ratio = 0.2

non_accident_train_set, non_accident_test_set, non_accident_val_set, train_count, test_count, val_count = split_dataset(filenames,
                                                                                  train_ratio,
                                                                                  test_ratio)

print("Train set size:", len(non_accident_train_set))
print("Test set size:", len(non_accident_test_set))
print("Val set size:", len(non_accident_val_set))
print("Train set size (in samples):", train_count)
print("Test set size (in samples):", test_count)
print("Val set size (in samples):", val_count)

['Numan_-334-_jpg.rf.396b382711b7552da9c5685a5b0e9e49.jpg', 'Navid_595_JPG.rf.d49c819ea4319fb42ed18602a3289a06.jpg', 'Navid_132_jpg.rf.f059877bf59264fc2cab8342177cf498.jpg', 'Pias--76-_jpg.rf.4232fd1bf6cf0df19e4e387e260c1f27.jpg', '152_jpg.rf.2897b85800196503e29149303ec7c454.jpg', '180_jpg.rf.0cd695fa4a4a07608737d1b41b05e4fe.jpg', 'Dipto_726_jpg.rf.8850fbed42887ef0795165389fe183f7.jpg', 'Dipto_790_jpg.rf.f384d4da12809fe1193c0e39713fd006.jpg', '186_jpg.rf.0f75c53b0801d51f83031a1490096717.jpg', 'Numan_-299-_JPG.rf.ccd4d4d41eba692c12b3006c0f274766.jpg', 'Dipto_457_jpg.rf.2ed0191cf6aa022c57b960fcea956633.jpg', 'Navid_494_jpg.rf.9670255b311c8c6af2448a02220ef30f.jpg', 'Dipto_472_jpg.rf.c1ae05ac13722e2fa4df0a0c3d52b641.jpg', 'Numan_-206-_jpg.rf.79c26b1fdee36792c0e2e8abe476d85b.jpg', 'Numan_-330-_jpg.rf.614fb158aef195678deafd66e3819d78.jpg', 'Pias--496-_jpg.rf.7b68af90a0ae2741780592e895b2a45b.jpg', 'Dipto_29_jpg.rf.5d8f222c7998067401c9b1fb80e59525.jpg', 'Dipto_834_jpg.rf.a88ecc6767ca7c4669a3fb

In [61]:
# move files to train, test, val directories
for filename in non_accident_train_set:
    full_file_path = os.path.join(non_accident_images_path, filename)
    if not os.path.exists(full_file_path):
        print("File does not exist:", full_file_path)
        continue
    shutil.move(full_file_path, os.path.join(dataset_train_path, "Non Accident", filename))

for filename in non_accident_test_set:
    full_file_path = os.path.join(non_accident_images_path, filename)
    if not os.path.exists(full_file_path):
        print("File does not exist:", full_file_path)
        continue
    shutil.move(full_file_path, os.path.join(dataset_test_path, "Non Accident", filename))

for filename in non_accident_val_set:
    full_file_path = os.path.join(non_accident_images_path, filename)
    if not os.path.exists(full_file_path):
        print("File does not exist:", full_file_path)
        continue
    shutil.move(full_file_path, os.path.join(dataset_val_path, "Non Accident", filename))

In [ ]:
what_to_delete = ['../../datasets/road-vehicle-images/', '../../datasets/CADP/', '../../datasets/1/']

for path in what_to_delete: 
    if os.path.exists(path):
        shutil.rmtree(path)
        print("Deleted:", path)

Deleted: ../../datasets/road-vehicle-images/
Deleted: ../../datasets/CADP/
Deleted: ../../datasets/1/
